## MQTT Subscriber

In this tutorial, I would like to tell how to implement a MQTT subscriber, which subscribes for a certain type of data from MQTT broker. 

I would like to create a hypothetical smart city, which clients publish below data:

```
- temperature
    - celcius
    - fahrenheit
- air pollution
    - CO level
    - SO2 level
- traffic speed
    - km/h
```

Subscribers only work on determined data types, a single purpose clients to collect data and do what ever they want. Like, show on the map, analyze etc.

Our topic structure is as shown:

```
<system_name>/<city_id>/<street_id>/<sensor_data>/<sub_data>
```

We want our subscriber to collect certain data for its own purpose. Since our topic structure is known by all systems, clients can be configured to parse data and collect certain information.

Using the topic info, a subscriber can identify id of city, street and sensor data types. 

This information can also be embedded into payload, that's the choice of design. 



In [ ]:
# let's check if paho-mqtt has been installed
!pip install paho-mqtt

In [ ]:
# import paho client
import paho.mqtt.client as mqtt

In [ ]:
# Define callback functions for Subscriber
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    
def on_message(client, userdata, msg):
    message = msg.payload.decode("utf-8")

    message_data = message.split('\t')
    timestamp_utc = message_data[0]
    data = message_data[1]
    
    topic_data = msg.topic.split('/')
    pollution_type = topic_data[len(topic_data) - 1]
    
    if pollution_type == 'no2_level':
        print('City: {}, Street {}, NO2 Level {} ug/m3 | {}'.format(topic_data[1], topic_data[2], message_data[1], timestamp_utc))
    else:
        print('City: {}, Street {}, SO2 Level {} ug/m3 | {}'.format(topic_data[1], topic_data[2], message_data[1], timestamp_utc))

In [ ]:
mqtt_server_ip = "172.17.0.2"
server_port = 1883

# uniqie client id
client_id = 'air_pollution_analyzer'

In [ ]:
system_name = "hypo_city"
air_pollution = 'air_pollution'

client = mqtt.Client(client_id)

client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_server_ip, server_port, keepalive=60)

# I will get, all cities and all cities streets 
subscribe_topic = system_name + '/+/+/air_pollution/+'

client.subscribe(subscribe_topic, qos=0)

In [ ]:
# start loop forever to subscribe wanted topic
client.loop_forever()